# Wiskundige technieken les 3

Notebook bij les 3 van de leerlijn wiskundige technieken van S3 - AI. 

© Auteur: Rianne van Os

In de vorige lessen hebben we gewerkt met functies met maar 1 input-variabele. Als je machine learning modellen gaat trainen zul je altijd meer dan 1 variabele hebben. In deze les introduceren we multivariabele functies en leer je hoe je deze moet differentiëren om ook hier gradient descent op toe te kunnen passen.

**Voorbereiding voor de les:**
- Neem theorie en opdrachten uit dit notebook door t/m opdracht 3.2


In [ ]:
#import benodigde libraries
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from typing import Callable

## Multivariabele functies

We hebben tot nu toe steeds gekeken naar functies met 1 input variabele, maar in de praktijk zullen de functies die we willen minimaliseren altijd uit meerdere variabelen bestaan. In code ben je al gewend hiermee te werken, maar wiskundig is dit misschien nieuw voor je. We bekijken eerst een simpel voorbeeld, namelijk het berekenen van de som van 2 input. In code ziet dat er zo uit:

In [ ]:
def sum_of_two_values(x: float, y: float) -> float:
    return x + y

Wiskundig kun je dit alsvolgt noteren:
$f(x,y) = x + y$

Merk op dat als we dit als lambda-functie definieren in python, het al veel meer uitziet als de wiskundige notatie.

In [ ]:
f = lambda x, y: x+ y

We gebruiken hier de variabele namen $x$ en $y$, maar deze kunnen in principe vanalles zijn. We hadden ook $f(a,b) =  a+ b$ kunnen schrijven, of een andere veelvoorkomende notatie is $f(x_1, x_2) = x_1 + x_2$.
In de machine learning kom je ook vaak griekse letters tegen, bijvoorbeeld $f(\omega_1, \omega_2) = \omega_1 + \omega_2$. (Spreek $\omega$ uit als omega)

We kunnen nu de waarde van $f$ uitrekenen op basis van input-waarden $x$ en $y$. Bijvoorbeeld:
$$f(2, 3) = 2 + 3 = 5$$
$$f(-0.1,2.8) = -0.1 + 2.8 = 2.7$$

Als je de grafiek van de functie $f(x,y)$ wilt bekijken, heb je 3 dimensies nodig. De x-as en y-as representeren de input, de z-as de output. We gebruiken plotly om een interactieve visualisatie hiervan te maken. (bron: https://plotly.com/python/3d-surface-plots/)

In [ ]:
# Maak een grid van alle x, y waarden op een interval van -5 tot 5
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

#bereken z = f(x, y) voor alle x, y
z = f(X, Y)

fig = go.Figure(
    data = go.Surface(
        x=X,
        y=Y,
        z=z,
        contours={
            "x": {"show": True, "color":"black", "width": 1},
            "y": {"show": True, "color":"black", "width": 1},
            "z": {"show": False}
        }
    )
)

fig.update_layout(
    title = "Grafiek van de functie f(x, y) = x + y",
    scene = dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='f(x, y)',
    )
)

fig.show()


Dit is een beetje een saai voorbeeld, de grafiek van deze functie is namelijk een plat vlak. Laten we een interessantere functie bekijken:
$$ g(x, y) = x^2 + 2y^2

Een plot van deze functie laat zien dat de bijbehorende grafiek een paraboloide is met één minimum.

In [ ]:
# Maak een grid van alle x, y waarden op een interval van -5 tot 5
g = lambda x, y: x**2+2*y**2
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

#bereken z = f(x, y) voor alle x, y
z = g(X, Y)

#plot 3D grafiek met plotly
fig = go.Figure(data=go.Surface(z=z, x=x, y=y, contours={
            "x": {"show": True, "color":"black", "width": 1},
            "y": {"show": True, "color":"black", "width": 1},
            "z": {"show": False}
        }))

fig.update_layout(
    title='Grafiek van de functie $g(x, y) = x^2 + 2y^2$',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    )
)
fig.show()

We zien de paraboloide met een minimum in het punt $(0,0)$. Ook kun je zien dat de grafiek in de y-richting 2 keer zo snel stijgt als in de x-richting.

### Opdracht 3.1
Bereken $g(1,2)$, $g(5,5)$, $g(0,0)$ en $g(5, -5)$ en controleer je antwoord door het punt op te zoeken in bovenstaande grafiek.


## Partieel differentiëren
Ook van deze grafiek kunnen we de helling (of gradient) in een punt bepalen, alleen hebben we dan te maken de helling in 2 richtingen, namelijk in de $x$-richting en in de $y$-richting. We zullen dus ook 2 afgeleiden moeten bepalen.
De afgeleide $\frac{df}{dx}$ bepaalt de helling in de x-richting, de afgeleide $\frac{df}{dy}$ geeft de helling in de y-richting. Bij multivariabele functies noemen we het bepalen van deze afgeleiden *partieel differentiëren*.

Bij het bepalen van $\frac{df}{dx}$ mogen we $y$ als een constante beschouwen en bij het bepalen van $\frac{df}{dy}$ beschouwen we $x$ als een constante. 

We krijgen dan, als:
$f(x,y) = x^2 + 2 y^2$
Dan is:
$$\frac{df}{dx} = 2x$$
$$\frac{df}{dy} = 4y$$

Nu kunnen we de gradient in een gegeven punt bepalen. Nemen we bijvoorbeeld $x=2$ en $y=2$ dan is:
$$\frac{df}{dx}(2,2) = 2\cdot 2 = 4$$
$$\frac{df}{dy}(2,2) = 4 \cdot 2 = 8$$

De verandering in de x-richting is dus 4, en in de y-richting is die 8. Kijk of je in de 3D plot terug kunt zien dat de grafiek in de buurt van het punt (2,2) sneller stijgt in de y-richting dan in de x-richting.

Een andere notatie voor deze gradient is $\nabla f = [2x, 4y]$. Om aan te geven wat de gradient is in een gegeven punt, bijvoorbeeld in (2,2), gebruik je de notatie $\nabla f(2,2) = [4,8]$.

### Opdracht 3.2
Gegeven is de functie $g(x,y) = (x-y)^2$ 

a. Maak een plot van $g(x,y)$ door de code hierboven te kopieren en aan te passen. Bepaal de gradient $\nabla g$. Oftewel, bepaal $\frac{dg}{dx}$ en $\frac{dg}{dy}$.

b. Bereken de gradient op de punten $(1,1)$, $(1,2)$ en $(-1,-1)$, oftewel bereken $\nabla g(1,1)$, $\nabla g(1,2)$ en $\nabla g (-1,-1)$.


## Gradient descent bij 2-dimensionale functie

Om het minimum van een 2-dimensionale functie te vinden kunnen we ook hier gradient descent op toepassen. Dan moeten we in iedere stap voor beide variabelen bepalen in welke richting we een stap moeten zetten om bij het minimum uit te komen. Daarvoor hebben we dus de partieel afgeleiden in beide richtingen nodig. Hieronder zie je drie stappen gradient descent voor de functie $g(a,b) = (a+2b)^2$ in het startpunt $(a_0, b_0) = (1,1)$ met learning rate 0.1.

Eerst bepalen $\nabla g(a, b) = [2(a+2b), 4(a+2b)]$, waarmee we de helling in startpunt $(1,1)$ uit kunnen rekenen. Dat is $\nabla g(1,1) = [6, 12]$. Die $6$ gebruiken we om in de a-richting een naar het minimum te zetten, en de $12$ gebruiken we om in de b-richting een stap naar het minimum te zetten. Je krijgt dan:

- $a_1 = 1 - 0.1 \cdot 6 = 0.4$
- $b_1 = 1 - 0.1 \cdot 12 = 0.8$

Het nieuwe punt is dus $(0.4, 0.8)$. Ook hier berekenen we de gradient, en dat is $\nabla g(0.4, 0.8) = [2(0.4+2\cdot 0.8), 4(0.4+2\cdot 0.8)] = [5.6, 11.2]$. We gebruiken deze gradient om opnieuw een stap naar het minimum te zetten:
- $a_2 = 0.4 - 0.1 \cdot 5.6 = 0.4 - 0.56 = -0.16$
- $b_2 = 0.8 - 0.1 \cdot 11.2 = 0.8 - 1.12 = -0.32$

Het nieuwe punt is dus $(-0.16, -0.32)$. Dit kun je herhalen tot je (dicht genoeg) bij het minimum bent.

### Opdracht 3.3
Gegeven is de functie $g(x,y) = (x-y)^2$ en het startpunt $(x_0, y_0) = (3,2)$. Bereken handmatig twee stappen van het gradient descent algoritme (zoals hierboven voorgedaan is). Gebruik een learning rate van 0.1. Wat zijn de nieuwe waarden voor $x$ en $y$ na deze twee stappen? Wat is de waarde van $g(x,y)$ in dit nieuwe punt?

### Het gradient descent algoritme voor 2-dimensionale functies
Hierboven zag je een concreet voorbeeld van gradient descent op een 2-dimensionale functie, hieronder zie je het algoritme in het algemene geval:

Om het minimum van een functie $f(a,b)$ te vinden, doe je het volgende: 

1. Kies willekeurige startwaarden voor $a$ en $b$, deze noemen we $a_0$, en $b_0$

2. Bepaal $\nabla f(a_0, b_0)$, oftewel $\frac{df}{da}(a_0, b_0)$ en $\frac{df}{db}(a_0, b_0)$

3. Defineer: 
    - $a_1 = a_0 - lr \cdot \frac{df}{da}(a_0, b_0)$

    - $b_1 = b_0 - lr \cdot \frac{df}{db}(a_0, b_0)$

Herhaal stap 2 en 3. In stap $i+1$ doe je:

2. Bereken $\nabla f(a_i, b_i)$,

3. Definieer $a_{i+1} = a_i - lr \cdot \frac{df}{da}(a_i, b_i)$ en $b_{i+1} = b_i - lr \cdot \frac{df}{db}(a_i, b_i)$

Waarbij $lr$ weer staat voor de learning rate. Uiteraard moet je bij het implementeren weer stopcriteria, zoals   een maximaal aantal stappen zijn en een threshold voor $\frac{df}{da}(a_i,b_i)$ en $\frac{df}{db}(a_i,b_i)$.

### Opdracht 3.4
Gegeven is de functie $f(x,y) = x^2 + y^2 + 2x + 4y + 1$. 

a.  Bereken het minimum van de grafiek door de partieel afgeleiden $\frac{df}{dx}$ en $\frac{df}{dy}$ gelijk te stellen aan nul. Wat zijn de bijbehorende waarden voor $x$ en $y$ en wat is de waarde van $f(x,y)$ in dit minimum?

b.  Maak een plot van de grafiek van $f(x,y)$ en markeer het minimum.

c.  Schrijf nu een pythonfunctie genaamd *gradient_descent_2d* om met gradient descent het minimum van de functie $f(x,y) = x^2 + y^2 + 2x + 4y + 1$ te vinden. Experimenteer met verschillende learning rates, startpunten en stopcriteria. Zoek naar waarden waarvoor het algoritme goed werkt, maar ook naar waarden waarvoor het niet werkt.

## Contourplot om de gradient stappen te visualiseren
Om de stappen van gradient descent te visualiseren bij een 2-dimensionale functie kunnen we een contourplot gebruiken. Dit is een 2-dimensional plot waarin hoogtelijnen de functie-waarden aangeven. Hieronder een contourplot van de functie $f(x,y) = x^2 + y^2$.

In [ ]:
# Definieer functie
f = lambda x, y: x**2 + y**2

# Maak een meshgrid van x en y waarden
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

# Bereken bijbehorende z waarden
z = f(X, Y)

#maak contourplot
fig = go.Figure(data=go.Contour(z=z,x = x,y=y))

fig.show()

Je kunt hier ook de stappen van gradient descent in tonen. Stel we hadden als startpunt $(4,4)$ en gradient descent gaf ons de volgende stappen:
a_array = [4,3,2,1,0] en b_array = [4,3,2,1,0] (Merk op, dit voorbeeld is fictief, enkel zodat we kunnen laten zien hoe je dit plot.) Dan kun je dit alsvolgt visualiseren:

In [ ]:
# Definieer functie
f = lambda x, y: x**2 + y**2

# Maak een meshgrid van x en y waarden
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

# Bereken bijbehorende z waarden
z = f(X, Y)

x_array = [4,3,2,1,0]
y_array = [4,3,2,1,0]

#maak contourplot
fig = go.Figure(data=go.Contour(z=z,x = x,y=y))

# Add line for the path
fig.add_trace(
        go.Scatter(
                x=x_array,
                y=y_array,
                mode='lines+markers',
                showlegend=False
))

# Add marker for the starting point
fig.add_trace(
        go.Scatter(
                x=[x_array[0]],
                y=[y_array[0]],
                mode='markers',
                marker=dict(size=8, symbol='circle'),
                name=f'Start: ({x_array[0]},{y_array[0]})'
))

# Add marker for the end point
fig.add_trace(
        go.Scatter(
                x=[x_array[-1]],
                y=[y_array[-1]],
                mode='markers',
                marker=dict(size=8, symbol='star'),
                name=f'End: ({x_array[-1]},{y_array[-1]})'
))

fig.show()

### Opdracht 3.4
Maak een contourplot van de functie $f(x,y) = x^2 + y^2 + 2x + 4y + 1$ en markeer de stappen van gradient descent die je gevonden hebt met je functie *gradient_descent_2d*.

## Lokale en globale minima
In het voorbeeld hierboven zagen we een functie met 1 minimum. Met gradient descent kan dit minimum relatief eenvoudig gevonden worden. In het 1-dimensionale geval hadden we al gezien dat het ook voor kan komen dat een functie meerdere minima heeft. Hiervan kunnen er sommige lokaal en andere globaal zijn. Dan kan het van je learning rate en startwaarden afhangen welk minimum je vindt. 

### Opdracht 3.5

Onderstaande 2 functies horen bij grafieken met meerdere minima. Maak van beide grafieken een plot en probeer met gradient de verschillende minima te vinden. Hiervoor zul je moeten varieren met de startwaarden en mogelijk ook met de learning rate. Maak steeds een contourplot om te visualiseren welk minimum je gevonden hebt. De functies waarvoor je dit moet doen zijn:
- $f(x_1, x_2) = (x_1^2+x_2-11)^2 + (x_1+x_2^2-7)^2$ (Deze functie staat bekend als de Himmelblau functie, een in de wiskunde veel gebruikte functie om optimalisatie algoritmes te testen)

- $g(x_1,x_2) = 0.5(x_1^4 - 16x_1^2 + 5x_1 + x_2^4 - 16x_2^2)$
